# Spark: Getting Started


## Step 0: Prerequisites & Installation

Follow [these instructions](https://docs.databricks.com/notebooks/notebooks-manage.html#import-a-notebook) to import this notebook into Databricks

Run these commands in your terminal (just once) if you want to run Spark locally.

 * These instructions require a Mac with [Anaconda3](https://anaconda.com/) and [Homebrew](https://brew.sh/) installed.
 * Useful for small data only. For larger data, try [Databricks](https://databricks.com/).

```bash
# Make Homebrew aware of old versions of casks
brew tap caskroom/versions

# Install Java 1.8 (OpenJDK 8)
brew cask install adoptopenjdk8

# Install the current version of Spark
brew install apache-spark

# Install Py4J (connects PySpark to the Java Virtual Machine)
pip install py4j

# Add JAVA_HOME to .bash_profile (makes Java 1.8 your default JVM)
echo "\n# Apache Spark\nexport JAVA_HOME=$(/usr/libexec/java_home -v 1.8)" >> ~/.bash_profile

# Add SPARK_HOME to .bash_profile
echo "\nexport SPARK_HOME=/usr/local/Cellar/apache-spark/2.4.5/libexec" >> ~/.bash_profile

# Add PySpark to PYTHONPATH in .bash_profile
echo "\nexport PYTHONPATH=$SPARK_HOME/python:$PYTHONPATH" >> ~/.bash_profile

# Update current environment
source ~/.bash_profile

```

## Step 1: Create a SparkSession with a SparkContext

In [4]:
import pyspark

# un-comment the following lines if you are running Spark locally
# spark = pyspark.sql.SparkSession.builder.getOrCreate()
# sc = spark.sparkContext

In [5]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [6]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v2.4.4 
 Master 
 local[8] 
 AppName 
 Databricks Shell

If we need a broadcast (i.e. global) variable, we can declare it like so:

In [8]:
glob = sc.broadcast(list(range(1, 3)))
glob.value

Out[4]: [1, 2]

## Step 2: Download some Amazon reviews (Toys & Games)

In [10]:
# Data is already in the repo, but you can also get it with
# !wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz
# !gunzip reviews_Toys_and_Games_5.json.gz

Follow [these instructions](https://docs.databricks.com/data/data.html#import-data-1) to import `reviews_Toys_and_Games_5.json` into Databricks

## Step 3: Create a Spark DataFrame

In [13]:
# this file path will be different if you are running Spark locally
df = spark.read.json('/FileStore/tables/reviews_Toys_and_Games_5.json')

In [14]:
df.persist()

Out[6]: DataFrame[asin: string, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, summary: string, unixReviewTime: bigint]

This last command, `.persist()`, simply stores the DataFrame in memory. See [this page](https://unraveldata.com/to-cache-or-not-to-cache/). It is similar to `.cache()`, but actually more flexible than the latter since you can specify which storage level you want. See [here](https://stackoverflow.com/questions/26870537/what-is-the-difference-between-cache-and-persist).

In [16]:
type(df)

Out[7]: pyspark.sql.dataframe.DataFrame

In [17]:
df.show(5) # default of 20 lines

+----------+-------+-------+--------------------+-----------+--------------+--------------+--------------------+--------------+
 asin|helpful|overall| reviewText| reviewTime| reviewerID| reviewerName| summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------+--------------------+--------------+
0439893577| [0, 0]| 5.0|I like the item p...|01 29, 2014|A1VXOAVRGKGEAK| Angie| Magnetic board| 1390953600|
0439893577| [1, 1]| 4.0|Love the magnet e...|03 28, 2014| A8R62G708TSCM| Candace|it works pretty g...| 1395964800|
0439893577| [1, 1]| 5.0|Both sides are ma...|01 28, 2013|A21KH420DK0ICA|capemaychristy| love this!| 1359331200|
0439893577| [0, 0]| 5.0|Bought one a few ...| 02 8, 2014| AR29QK6HPFYZ4| dcrm| Daughters love it| 1391817600|
0439893577| [1, 1]| 4.0|I have a stainles...| 05 5, 2014| ACCH8EOML6FN5| DoyZ|Great to have so ...| 1399248000|
+----------+-------+-------+--------------------+-----------+--------------+--------------+--------------------+--------------+
only showing top 5 rows

In [18]:
pdf = df.limit(5).toPandas()
pdf

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0439893577,"[0, 0]",5.0,I like the item pricing. My granddaughter want...,"01 29, 2014",A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600
1,0439893577,"[1, 1]",4.0,Love the magnet easel... great for moving to d...,"03 28, 2014",A8R62G708TSCM,Candace,it works pretty good for moving to different a...,1395964800
2,0439893577,"[1, 1]",5.0,Both sides are magnetic. A real plus when you...,"01 28, 2013",A21KH420DK0ICA,capemaychristy,love this!,1359331200
3,0439893577,"[0, 0]",5.0,Bought one a few years ago for my daughter and...,"02 8, 2014",AR29QK6HPFYZ4,dcrm,Daughters love it,1391817600
4,0439893577,"[1, 1]",4.0,I have a stainless steel refrigerator therefor...,"05 5, 2014",ACCH8EOML6FN5,DoyZ,Great to have so he can play with his alphabet...,1399248000


In [19]:
type(pdf)

Out[10]: pandas.core.frame.DataFrame

In [20]:
df.count()

Out[11]: 167597

In [21]:
df.columns

Out[12]: ['asin',
 'helpful',
 'overall',
 'reviewText',
 'reviewTime',
 'reviewerID',
 'reviewerName',
 'summary',
 'unixReviewTime']

In [22]:
df.printSchema()

root
-- asin: string (nullable = true)
-- helpful: array (nullable = true)
 |-- element: long (containsNull = true)
-- overall: double (nullable = true)
-- reviewText: string (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- reviewerName: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: long (nullable = true)

The 'nullable = true' bit means that the relevant column tolerates null values.

In [24]:
df.describe().show()

+-------+--------------------+------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
summary| asin| overall| reviewText|reviewTime| reviewerID| reviewerName| summary| unixReviewTime|
+-------+--------------------+------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
 count| 167597| 167597| 167597| 167597| 167597| 166759| 167597| 167597|
 mean|2.1290876613696043E9| 4.356307093802395| null| null| null| 474.1111111111111| 600.0|1.3487585072095563E9|
 stddev|2.1305790004603045E9|0.9935012992132014| null| null| null| 517.0615833762786| NaN| 6.089580845001178E7|
 min| 0439893577| 1.0| |01 1, 2001|A012468118FTQAINE...| | ! Just Junk !| 964742400|
 max| B00LBI9BKA| 5.0|~the RED "B/2" bl...|12 9, 2013| AZZYW4YOE1B6E|~purplemoon~ "The...|~My 9 year old sp...| 1406073600|
+-------+--------------------+------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+

In [25]:
df.describe('overall').show()

+-------+------------------+
summary| overall|
+-------+------------------+
 count| 167597|
 mean| 4.356307093802395|
 stddev|0.9935012992132014|
 min| 1.0|
 max| 5.0|
+-------+------------------+

In [26]:
reviews_df = df[['asin', 'overall']]

In [27]:
reviews_df.show()

+----------+-------+
 asin|overall|
+----------+-------+
0439893577| 5.0|
0439893577| 4.0|
0439893577| 5.0|
0439893577| 5.0|
0439893577| 4.0|
0439893577| 3.0|
0439893577| 3.0|
0439893577| 5.0|
0439893577| 4.0|
0439893577| 3.0|
0439893577| 5.0|
0439893577| 5.0|
0439893577| 3.0|
0439893577| 5.0|
0439893577| 5.0|
0439893577| 5.0|
0439893577| 5.0|
048645195X| 5.0|
048645195X| 4.0|
048645195X| 5.0|
+----------+-------+
only showing top 20 rows

In [28]:
def show(df, n=5):
    return df.limit(n).toPandas()

In [29]:
show(reviews_df)

,asin,overall
0,0439893577,5.0
1,0439893577,4.0
2,0439893577,5.0
3,0439893577,5.0
4,0439893577,4.0


In [30]:
reviews_df.count()

Out[20]: 167597

In [31]:
sorted_review_df = reviews_df.sort('overall')

In [32]:
show(sorted_review_df)

,asin,overall
0,0786955708,1.0
1,0976990709,1.0
2,0963679600,1.0
3,0786955708,1.0
4,0974665207,1.0


In [33]:
import pyspark.sql.functions as F

In [34]:
counts = reviews_df.agg(F.countDistinct('overall'))

In [35]:
counts.show()

+-----------------------+
count(DISTINCT overall)|
+-----------------------+
 5|
+-----------------------+

In [36]:
query = """
SELECT overall, COUNT(*)
FROM reviews
GROUP BY overall
ORDER BY overall
"""

In [37]:
reviews_df.createOrReplaceTempView('reviews')

In [38]:
output = spark.sql(query)

In [39]:
show(output)

,overall,count(1)
0,1.0,4707
1,2.0,6298
2,3.0,16357
3,4.0,37445
4,5.0,102790


In [40]:
output.collect()

Out[30]: [Row(overall=1.0, count(1)=4707),
 Row(overall=2.0, count(1)=6298),
 Row(overall=3.0, count(1)=16357),
 Row(overall=4.0, count(1)=37445),
 Row(overall=5.0, count(1)=102790)]

In [41]:
reviews_df.count() - sum(output.collect()[i][1] for i in range(5))

Out[31]: 0

In [42]:
type(reviews_df)

Out[32]: pyspark.sql.dataframe.DataFrame

Convert to RDD!

In [44]:
reviews_df.rdd

Out[33]: MapPartitionsRDD[2224] at javaToPython at NativeMethodAccessorImpl.java:0

In [45]:
type(reviews_df.rdd)

Out[34]: pyspark.rdd.RDD

### Count the words in the first row

In [47]:
row_one = df.first()

In [48]:
row_one

Out[36]: Row(asin='0439893577', helpful=[0, 0], overall=5.0, reviewText='I like the item pricing. My granddaughter wanted to mark on it but I wanted it just for the letters.', reviewTime='01 29, 2014', reviewerID='A1VXOAVRGKGEAK', reviewerName='Angie', summary='Magnetic board', unixReviewTime=1390953600)

In [49]:
def word_count(text):
    return len(text.split())

In [50]:
word_count(row_one['reviewText'])

Out[38]: 20

In [51]:
from pyspark.sql.types import IntegerType

#'udf' is for User Defined Function!

word_count_udf = F.udf(word_count, returnType=IntegerType())

In [52]:
review_text_col = df['reviewText']

In [53]:
counts_df = df.withColumn('wordCount', word_count_udf(review_text_col))

In [54]:
# Remember that we set the default number of lines to show at 5.

show(counts_df).T

,0,1,2,3,4
asin,0439893577,0439893577,0439893577,0439893577,0439893577
helpful,"[0, 0]","[1, 1]","[1, 1]","[0, 0]","[1, 1]"
overall,5,4,5,5,4
reviewText,I like the item pricing. My granddaughter want...,Love the magnet easel... great for moving to d...,Both sides are magnetic. A real plus when you...,Bought one a few years ago for my daughter and...,I have a stainless steel refrigerator therefor...
reviewTime,"01 29, 2014","03 28, 2014","01 28, 2013","02 8, 2014","05 5, 2014"
reviewerID,A1VXOAVRGKGEAK,A8R62G708TSCM,A21KH420DK0ICA,AR29QK6HPFYZ4,ACCH8EOML6FN5
reviewerName,Angie,Candace,capemaychristy,dcrm,DoyZ
summary,Magnetic board,it works pretty good for moving to different a...,love this!,Daughters love it,Great to have so he can play with his alphabet...
unixReviewTime,1390953600,1395964800,1359331200,1391817600,1399248000
wordCount,20,22,76,31,47


In [55]:
from pyspark.sql.types import IntegerType
word_count_udf = F.udf(word_count, IntegerType())

# Registering our word_count() function so that we
# can use it with SQL! See documentation here:
# https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-UDFRegistration.html

df.createOrReplaceTempView('reviews')
spark.udf.register('word_count', word_count_udf)

Out[43]: <function __main__.word_count(text)>

In [56]:
query = """
SELECT asin, overall, reviewText, word_count(reviewText) AS wordCount
FROM reviews
"""

In [57]:
counts_df = spark.sql(query)

In [58]:
show(counts_df)

,asin,overall,reviewText,wordCount
0,0439893577,5.0,I like the item pricing. My granddaughter want...,20
1,0439893577,4.0,Love the magnet easel... great for moving to d...,22
2,0439893577,5.0,Both sides are magnetic. A real plus when you...,76
3,0439893577,5.0,Bought one a few years ago for my daughter and...,31
4,0439893577,4.0,I have a stainless steel refrigerator therefor...,47


In [59]:
def count_all_the_things(text):
    return [len(text), len(text.split())]

In [60]:
from pyspark.sql.types import ArrayType, IntegerType
count_udf = F.udf(count_all_the_things, returnType=ArrayType(IntegerType()))

In [61]:
counts_df = df.withColumn('counts', count_udf(df['reviewText']))

In [62]:
show(counts_df, 1)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,counts
0,0439893577,"[0, 0]",5.0,I like the item pricing. My granddaughter want...,"01 29, 2014",A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600,"[100, 20]"


In [63]:
slim_counts_df = (
    df.drop('reviewTime', 'helpful')
#       .drop('helpful')
      .withColumn('counts', count_udf(df['reviewText']))
      .drop('reviewText')
)

In [64]:
show(slim_counts_df, n=1)

,asin,overall,reviewerID,reviewerName,summary,unixReviewTime,counts
0,0439893577,5.0,A1VXOAVRGKGEAK,Angie,Magnetic board,1390953600,"[100, 20]"


In [65]:
aggs = counts_df.groupBy('reviewerID').agg({'overall': 'mean'})
aggs.collect()

Out[53]: [Row(reviewerID='A3R9X003XW0LNR', avg(overall)=4.36),
 Row(reviewerID='A2TZT5AMK1XQTC', avg(overall)=4.2),
 Row(reviewerID='A2UGBQSNS1I37M', avg(overall)=4.857142857142857),
 Row(reviewerID='AUAX1QWUCYKSX', avg(overall)=4.090909090909091),
 Row(reviewerID='A1XU4ZI0NUZ68M', avg(overall)=3.8333333333333335),
 Row(reviewerID='A2OIGPPBTR65MR', avg(overall)=4.421052631578948),
 Row(reviewerID='A2B2OT465SEI15', avg(overall)=4.833333333333333),
 Row(reviewerID='A3RSDKOJ28VU6B', avg(overall)=5.0),
 Row(reviewerID='A8SHD09Z1RI3', avg(overall)=4.0),
 Row(reviewerID='A1919BYRFSG18U', avg(overall)=4.666666666666667),
 Row(reviewerID='A10AKE9TAADHVV', avg(overall)=4.142857142857143),
 Row(reviewerID='A239YR4IQ0QAFY', avg(overall)=5.0),
 Row(reviewerID='A1VLE2SH9J8WYS', avg(overall)=4.785714285714286),
 Row(reviewerID='A24WSD48Z3WUZS', avg(overall)=4.5),
 Row(reviewerID='A17A1KTVI3DG6U', avg(overall)=3.5714285714285716),
 Row(reviewerID='A2GF3I3279PEXO', avg(overall)=4.714285714285714),
 Row(reviewerID='A2IKUVB2PS4QLT', avg(overall)=5.0),
 Row(reviewerID='APVMWJNB77FNP', avg(overall)=4.2),
 Row(reviewerID='A1VHPPBMKS6U1N', avg(overall)=4.4),
 Row(reviewerID='A2P2QMTLWLUQUZ', avg(overall)=3.75),
 Row(reviewerID='A6GMEO3VRY51S', avg(overall)=4.166666666666667),
 Row(reviewerID='A1IJ4Z3204XZ6N', avg(overall)=4.428571428571429),
 Row(reviewerID='A1UQNPIQ52PYSN', avg(overall)=2.8),
 Row(reviewerID='A3SKNXQJ357GEW', avg(overall)=4.166666666666667),
 Row(reviewerID='AJCK6OKNWGIYG', avg(overall)=5.0),
 Row(reviewerID='A12WD8M1UUA93G', avg(overall)=4.666666666666667),
 Row(reviewerID='AW9FZO9WF88CV', avg(overall)=4.285714285714286),
 Row(reviewerID='A2A56MRVND9JVW', avg(overall)=4.833333333333333),
 Row(reviewerID='A2TWO5CLBUA5E6', avg(overall)=4.166666666666667),
 Row(reviewerID='A14DLCVBFBK7XV', avg(overall)=4.4),
 Row(reviewerID='A1CEO7S4330SW2', avg(overall)=5.0),
 Row(reviewerID='AF89IW5TVF8I8', avg(overall)=4.444444444444445),
 Row(reviewerID='A2G0Y6FVP84EX7', avg(overall)=4.0),
 Row(reviewerID='A1BOJE4E44MYW6', avg(overall)=4.166666666666667),
 Row(reviewerID='A3EJOBNMT20Q6W', avg(overall)=5.0),
 Row(reviewerID='A1VSD1TTO9LTA6', avg(overall)=5.0),
 Row(reviewerID='A17FQHN25ETAQH', avg(overall)=4.5),
 Row(reviewerID='A3CPL126VL12H8', avg(overall)=4.666666666666667),
 Row(reviewerID='A29XAP7W7YG0ZA', avg(overall)=4.833333333333333),
 Row(reviewerID='A2URD460P4XY10', avg(overall)=5.0),
 Row(reviewerID='A3MSH2YW203752', avg(overall)=5.0),
 Row(reviewerID='ARIFCL50JD5SK', avg(overall)=4.545454545454546),
 Row(reviewerID='A28973L3OU3OVV', avg(overall)=3.4),
 Row(reviewerID='A3YAIGMXF3049', avg(overall)=4.0625),
 Row(reviewerID='A1CQD4YELHLXBB', avg(overall)=4.333333333333333),
 Row(reviewerID='A3UGLZNHE8MSNT', avg(overall)=4.4),
 Row(reviewerID='A1OPV6BD37T1VL', avg(overall)=4.090909090909091),
 Row(reviewerID='A2S0HZ2WHXRKD', avg(overall)=4.0),
 Row(reviewerID='A1TA9VB0SA3FEO', avg(overall)=4.0),
 Row(reviewerID='ARD9QTJXZXAC8', avg(overall)=3.1904761904761907),
 Row(reviewerID='A1NXRF3I791HRI', avg(overall)=3.875),
 Row(reviewerID='A2GCEBDTV8R0M2', avg(overall)=3.625),
 Row(reviewerID='A9XCW04U6X4NP', avg(overall)=4.4),
 Row(reviewerID='A13U56PF19VGSW', avg(overall)=5.0),
 Row(reviewerID='A2JPEYP1OTIQE4', avg(overall)=5.0),
 Row(reviewerID='A1PBLFIRAZII1F', avg(overall)=5.0),
 Row(reviewerID='A2TKEMX4M9Y6E6', avg(overall)=5.0),
 Row(reviewerID='A3QBCX2QMKJ7AH', avg(overall)=4.515151515151516),
 Row(reviewerID='A3Q1QHUK0JDLIE', avg(overall)=5.0),
 Row(reviewerID='A30Q5JUJULYNV8', avg(overall)=4.2),
 Row(reviewerID='A3N041QAKY7BVW', avg(overall)=3.0),
 Row(reviewerID='A1S9TVJIMBPB5Q', avg(overall)=4.833333333333333),
 Row(reviewerID='A3OPXNND84JPV6', avg(overall)=4.6),
 Row(reviewerID='A3LUUDFQJWLRTB', avg(overall)=4.0),
 Row(reviewerID='ARRAPMXS0U741', avg(overall)=5.0),
 Row(reviewerID='ATRQPL5OG4NIP', avg(overall)=4.2),
 Row(reviewerID='A375NF45A1MDWI', avg(overall)=4.083333333333333),
 Row(reviewerID='A3FCGR4EK16XUF', avg(overall)=5.0)

### A few more basic commands

Please refer also to the [official programming guide](http://spark.apache.org/docs/latest/rdd-programming-guide.html).

In [67]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

In [68]:
distData.

In [69]:
def multiply(a, b):
    return a * b

In [70]:
distData.reduce(multiply)

In [71]:
def subtract1(a, b):
    return a - b

In [72]:
distData.reduce(subtract1)

In [73]:
def subtract2(a, b):
    return b - a

In [74]:
distData.reduce(subtract2)

Can you explain these "subtraction" results?

In [76]:
distData.filter(lambda x: x < 4).collect()

### Reading files

```sc.textFile()``` for .txt files

`.toJSON()` for .json files

In [80]:
dfjson = counts_df.toJSON()

In [81]:
df2 = spark.read.json(dfjson)

In [82]:
df2.printSchema()

In [83]:
counts_df

In [84]:
type(df.toPandas())